In [1]:
import os
if 'jbook' in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../..")))
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# from importlib import reload  # Not needed in Python 2
# import logging
# reload(logging)
# logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

# Data Quality Assessment
The second stage of data processing is the Data Quality Assessment. This stage ensures that our dataset is ready for subsequent analysis and modeling tasks. By identifying and rectifying data quality issues early, we can avoid potential pitfalls that might compromise the integrity and accuracy of our results.

## Data Quality Checks
In this stage, we employ a series of tasks designed to identify and address any noise or irregularities within the dataset. Each task focuses on a specific aspect of data quality, ranging from detecting duplicate entries to identifying profanity, special patterns, and other potential sources of bias or distortion.
1. **Duplicate Rows**: We identify and remove duplicate entries to ensure that each observation is unique, preventing skewed analyses and inflated metrics.
2. **Null Values**: We detect and handle missing data appropriately, which could involve imputation, deletion, or flagging incomplete records for further investigation.
3. **Outliers**: Check for outliers in numeric columns using the non-parametric Interquartile Range (IQR) method.
4. **Non-English Text**: We check for and address non-English text in reviews and app names, as they may not be relevant to our analysis or could require special handling.
5. **Emojis**: Emojis can carry significant meaning in certain contexts but might also introduce noise. We identify and decide on their treatment—whether to retain, remove, or translate them into textual representations.
6. **Excessive Special Characters**: Special characters can disrupt text analysis and need to be managed, either by cleaning or encoding them appropriately.
7. **Invalid Dates**: We verify that date values fall within expected ranges and formats, correcting or flagging anomalies for further review.
8. **Invalid Ratings**: Ratings that fall outside the expected scale (e.g., 1 to 5) are identified and corrected or flagged.
9. **Profanity**: We detect and handle profane content to ensure that our dataset adheres to appropriate usage standards, especially if it's intended for public or sensitive applications.
10. **Special Patterns**: We identify and manage special patterns such as URLs, phone numbers, and emails. These patterns could be indicative of spam or need to be anonymized to protect privacy.

By conducting these data quality checks, we ensure that our dataset is clean, reliable, and ready for detailed analysis. This foundational step sets the stage for accurate insights and robust conclusions in the subsequent phases of our data processing pipeline.

In [3]:

from appinsight.data_prep.dqa import DataQualityAssessment, DQAConfig

We've encapsulated the data quality assessment process in a `DataQualityAssessment` class. Configured with source and target files, this class conducts the 10 data quality checks, marking the observations that require attention.

In [4]:
config = DQAConfig(force=True)
warnings.filterwarnings("ignore", category=FutureWarning)
dqa = DataQualityAssessment(config=config)
data = dqa.execute()

Dask dashboard available at: http://127.0.0.1:8787/status
Dask dashboard available at: http://127.0.0.1:37127/status


# ============================================================================ #
#                        DataQualityAssessment Pipeline                        #
# ============================================================================ #

Task ReadTask completed successfully. Runtime: 01 Minutes 00.951281 Seconds
Task DetectDuplicateRowTask completed successfully. Runtime: 02 Minutes 19.425275 Seconds
Task DetectDuplicateRowTask completed successfully. Runtime: 00 Minutes 12.360499 Seconds
Task DetectNullValuesTask completed successfully. Runtime: 00 Minutes 07.966793 Seconds
Task DetectOutliersTask completed successfully. Runtime: 00 Minutes 01.807617 Seconds
Task DetectOutliersTask completed successfully. Runtime: 00 Minutes 12.052286 Seconds


Exception occurred in DetectNonEnglishTask.execute_task called with <appinsight.data_prep.dqa.DetectNonEnglishTask object at 0x7f3e14243af0>, data=                  id      app_id                        app_name category_id  \
8082666   1119912682   302584613                   Amazon Kindle        6018   
12470837   599135993   377951542           Crackle - Movies & TV        6016   
8259344    817378711   379693831    Audible: Audio Entertainment        6018   
15167672  1140598740   454638411                       Messenger        6005   
17233515  5104781144   912561374    Marco Polo - Video Messenger        6005   
...              ...         ...                             ...         ...   
12569055  7129296333  1075603018                      Funimation        6016   
12708744  8063238669  1492683521  TuckerMoji - Tucker Budzyn Dog        6016   
20043110  4677288812   316800034                         Workday        6000   
16939968  8030630370  1269081011      Zoe: Lesbian Da

AttributeError: 'DetectNonEnglishTask' object has no attribute '_new_column'

Let's get a summary of the data quality issues by type.

In [ ]:
dqa.overview()

## Data Quality Impressions

The data quality assessment (DQA) conducted on the AppVoC dataset revealed several key issues that need to be addressed to ensure the integrity and reliability of the analysis. These issues include:

- **Duplicates**: A small percentage (<1%) have duplicate review ids.
- **Null Values**: Fortunately, there were no null values detected in the dataset.
- **Invalid Entries**: There were no invalid dates or invalid ratings found.
- **Outliers**: Outliers were identified in vote sums, and vote counts which *could* potentially distort analysis results.
- **Non-English**: A notable proportion of the app names (15%) and review content (4.2%) was flagged for being non-English.
- **Special Characters**: A small percentage of reviews were noted for the presence of special characters in excessive proportions.
- **Profanity & Sensitive Information**: Instances of profanity and the presence of email addresses, URLs, or phone numbers were minimal.

Given these findings, the next step is to identify and treat the data quality issues with the greatest potential negative impact on dataset integrity and downstream analyses. This may involve, without limitation:

- **Removing Duplicates**: Eliminating observations with duplicate review IDs.
- **Handling Outliers**: Identifying and appropriately managing outliers in vote sums, and vote counts.
- **Addressing Non-English Text**: Filtering or translating non-English reviews. 
- **filtering Noise**: Filtering or removing excessive special characters from reviews.
- **Ensuring Clean Content**: Censor or remove reviews containing profanity, and personal identifying information such as phone numbers, URLs, and email addresses.

Cue the action!